In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Файлы "Таблица студентов.csv" и "Изменения.csv" выгружены 7 июня 2022 года

In [2]:
df = pd.read_csv('Neto_stud.csv', sep='\t', encoding='utf-8')
df.columns=['N', 'Name', 'ID', 'LINK', 'AGE', 'MOB', 'MAIL', 'CITY', 'BU', 'C1', 'C2', 'COURSE', 'DATES', 'F', 'FIN', 'PTD', 'FIN_PTD']
df = df.drop(['N', 'LINK', 'MOB', 'MAIL', 'C1', 'C2', 'F', 'FIN', 'PTD', 'FIN_PTD'], axis=1)
df = df.replace('Нет сведений', np.nan)
df['AGE'] = df['AGE'].astype('float64')
df = df.drop_duplicates()

# Тут ёбана магия

In [2]:
BU_df =  pd.read_csv('Stud_orig.csv', sep='\t', encoding='utf-8')
BU_df
BU_df.columns=['N', 'Name', 'ID', 'LINK', 'AGE', 'MOB', 'MAIL', 'CITY', 'BU', 'C1', 'C2', 'COURSE', 'DATES', 'F', 'FIN', 'PTD', 'FIN_PTD', 'NoName']
BU_df =BU_df.drop(['N', 'LINK', 'MOB', 'MAIL', 'C1', 'C2', 'F', 'FIN', 'PTD', 'FIN_PTD', 'NoName'], axis=1)
BU_df = BU_df.replace('Нет сведений', np.nan)
BU_df['AGE'] = BU_df['AGE'].astype('float64')
BU_df = BU_df.drop_duplicates()

In [5]:
BU_df_1 = BU_df.set_index('ID')
BU_df_1 = BU_df_1.sort_values('DATES')
BU_df_1 = pd.DataFrame(BU_df.groupby('ID')['BU'].first())
BU_df_1.columns = ['FIRST_BU']

joined = BU_df_1.join(BU_df)
joined.groupby('FIRST_BU')['BU'].value_counts().to_frame()
heatmap = joined.groupby('FIRST_BU')['BU'].value_counts().to_frame().unstack('BU')

In [6]:
heatmap

BU                                                      \
BU                    B2G Аналитика Бизнес и управление Дизайн и UX Маркетинг   
FIRST_BU                                                                        
B2G                   2.0       3.0                 3.0         2.0       NaN   
Аналитика             5.0       NaN                 3.0         NaN       2.0   
Бизнес и управление   4.0       1.0                 5.0         3.0       9.0   
Дизайн и UX           2.0       1.0                 2.0         NaN       5.0   
Маркетинг            11.0       8.0                 9.0        10.0      15.0   
Программирование      3.0       1.0                 1.0         3.0       3.0   

                                      
BU                  Программирование  
FIRST_BU                              
B2G                              4.0  
Аналитика                        5.0  
Бизнес и управление              6.0  
Дизайн и UX                      7.0  
Маркетинг                       18.0  
Программирование                 8.0

In [7]:
with pd.ExcelWriter("BU_heatmap.xlsx") as writer:
    heatmap.to_excel(writer)

In [52]:
students_c = joined.groupby('ID')['COURSE'].unique().to_frame()
unknown = students_c['COURSE'].explode()
students = students_c.join(pd.crosstab(unknown.index, unknown))
students = students.drop('COURSE', axis=1)
students.sum(axis=1) > 2

# индексы всех студентов, которые взяли больше 2х курсов в Нетологии
students[students.sum(axis=1) > 2].index

Int64Index([    838,     883,    3895,    4962,    5604,    8011,    8330,
              13829,   19655,  272482,
            ...
            7661234, 7773373, 7791073, 7818091, 7851539, 7854811, 7874332,
            7897132, 7914876, 7969397],
           dtype='int64', name='ID', length=612)

# конец ёбаной магии

In [3]:
def start_date(row):
    return str(row).split(' - ')[0]

In [4]:
def end_date(row):
    return str(row).split(' - ')[-1]

In [5]:
df['SD'] = df['DATES'].apply(start_date)
df['ED'] = df['DATES'].apply(end_date)
df['SD'] = pd.to_datetime(df['SD'], format='%Y-%m-%d')
df['ED'] = pd.to_datetime(df['ED'], format='%Y-%m-%d')
df = df.drop('DATES', axis=1)
df['DURATION'] = df['ED'] - df['SD']

In [6]:
def age_range(age):
    if 0 <= age <= 18: return '0-18'
    elif 19 <= age <= 29: return '19-29'
    elif 30 <= age <= 39: return '30-39'
    elif 40 <= age <= 49: return '40-49'
    elif 50 <= age <= 59: return '50-59'
    elif 60 <= age <= 69: return '60-69'
    elif age >=70: return '70+'
    else: return '-'

In [7]:
df['AGE_RANGE'] = df['AGE'].apply(age_range)
df.head()

,Name,ID,AGE,CITY,BU,COURSE,SD,ED,DURATION,AGE_RANGE
0,Арина Шевяхова,6593669,25.0,NaN,Аналитика,Машинное обучение: фундаментальные инструменты...,2021-12-31,2022-04-03,93 days,19-29
1,Арина Шумилова,6555561,35.0,NaN,Маркетинг,SMM-менеджер: продвижение бизнеса в социальных...,2021-12-31,2022-04-03,93 days,30-39
2,Арина Щукарева,7372409,23.0,NaN,Маркетинг,Интернет-маркетолог: с нуля до middle. Специал...,2021-12-30,2022-07-02,184 days,19-29
3,Арина Щукарева,7372409,23.0,NaN,Маркетинг,Интернет-маркетолог: с нуля до middle. Специал...,2021-12-30,2022-05-02,123 days,19-29
4,Арина Щукарева,7372409,23.0,NaN,Маркетинг,Интернет-маркетолог: с нуля до middle. Специал...,2021-12-30,2022-05-02,123 days,19-29


In [8]:
# Выкорячиваем пол из ФИО

dfru = pd.read_csv('FN_ru.csv')

rumalenames = set(dfru[dfru['Gender'] == 'male']['GivenName'])
rumalesurnames = set(dfru[dfru['Gender'] == 'male']['Surname'])

rufemalenames = set(dfru[dfru['Gender'] == 'female']['GivenName'])
rufemalesurnames = set(dfru[dfru['Gender'] == 'female']['Surname'])

def gender(row):
    name = row.strip().split(' ')[0]
    surname = row.strip().split(' ')[-1]

    if name in rumalenames or surname in rumalesurnames:
        return 'м'

    elif name in rufemalenames or surname in rufemalesurnames:
        return 'ж'

    else:
        return np.nan

In [9]:
df['GENDER'] = df['Name'].apply(gender)

,Name,ID,AGE,CITY,BU,COURSE,SD,ED,DURATION,AGE_RANGE,GENDER
0,Арина Шевяхова,6593669,25.0,NaN,Аналитика,Машинное обучение: фундаментальные инструменты...,2021-12-31,2022-04-03,93 days,19-29,ж
1,Арина Шумилова,6555561,35.0,NaN,Маркетинг,SMM-менеджер: продвижение бизнеса в социальных...,2021-12-31,2022-04-03,93 days,30-39,ж
2,Арина Щукарева,7372409,23.0,NaN,Маркетинг,Интернет-маркетолог: с нуля до middle. Специал...,2021-12-30,2022-07-02,184 days,19-29,ж
3,Арина Щукарева,7372409,23.0,NaN,Маркетинг,Интернет-маркетолог: с нуля до middle. Специал...,2021-12-30,2022-05-02,123 days,19-29,ж
4,Арина Щукарева,7372409,23.0,NaN,Маркетинг,Интернет-маркетолог: с нуля до middle. Специал...,2021-12-30,2022-05-02,123 days,19-29,ж
...,...,...,...,...,...,...,...,...,...,...,...
57398,Ясмин Ачилова,7202942,19.0,NaN,Дизайн и UX,UX/UI-дизайнер,NaT,NaT,NaT,19-29,NaN
57399,ясмина,7403909,NaN,NaN,Аналитика,Аналитик BI,NaT,NaT,NaT,-,NaN
57400,Ясмина Ибадова,6865633,37.0,NaN,Маркетинг,Профессия PR в digital: управление коммуникаци...,NaT,NaT,NaT,30-39,NaN
57401,Ясон Тихонов,7195659,30.0,NaN,Программирование,DevOps-инженер,NaT,NaT,NaT,30-39,NaN


In [11]:
df.to_csv('neto_stud_final.csv', encoding='cp1251')

In [ ]:
# with pd.ExcelWriter("final.xlsx") as writer:
#     df.to_excel(df) 

In [15]:
fin = pd.read_csv('neto_stud_final.csv', encoding='cp1251')
fin = fin.drop(['Unnamed: 0', 'Name'], axis=1)
fin

,ID,AGE,CITY,BU,COURSE,SD,ED,DURATION,AGE_RANGE,GENDER
0,6593669,25.0,NaN,Аналитика,Машинное обучение: фундаментальные инструменты...,12/31/2021,4/3/2022,93 days,19-29,ж
1,6555561,35.0,NaN,Маркетинг,SMM-менеджер: продвижение бизнеса в социальных...,12/31/2021,4/3/2022,93 days,30-39,ж
2,7372409,23.0,NaN,Маркетинг,Интернет-маркетолог: с нуля до middle. Специал...,12/30/2021,7/2/2022,184 days,19-29,ж
3,7372409,23.0,NaN,Маркетинг,Интернет-маркетолог: с нуля до middle. Специал...,12/30/2021,5/2/2022,123 days,19-29,ж
4,7372409,23.0,NaN,Маркетинг,Интернет-маркетолог: с нуля до middle. Специал...,12/30/2021,5/2/2022,123 days,19-29,ж
...,...,...,...,...,...,...,...,...,...,...
46208,7202942,19.0,NaN,Дизайн и UX,UX/UI-дизайнер,NaN,NaN,NaN,19-29,ж
46209,7403909,NaN,NaN,Аналитика,Аналитик BI,NaN,NaN,NaN,-,ж
46210,6865633,37.0,NaN,Маркетинг,Профессия PR в digital: управление коммуникаци...,NaN,NaN,NaN,30-39,ж
46211,7195659,30.0,NaN,Программирование,DevOps-инженер,NaN,NaN,NaN,30-39,м


In [22]:
fin.loc[(fin['COURSE'] == 'тестовый набор') | (fin['COURSE'] == 'тестовый набор 2')]

,ID,AGE,CITY,BU,COURSE,SD,ED,DURATION,AGE_RANGE,GENDER
6694,5626596,29.0,Пенза,Прочее,тестовый набор 2,11/17/2021,7/13/2022,238 days,19-29,м
6703,6688553,NaN,NaN,Прочее,тестовый набор 2,11/17/2021,7/13/2022,238 days,-,м
10280,5464783,2.0,NaN,Прочее,тестовый набор 2,10/24/2021,7/25/2022,274 days,0-18,м
10281,5464783,2.0,NaN,Прочее,тестовый набор,10/24/2021,7/25/2022,274 days,0-18,м
23078,6283998,31.0,NaN,Прочее,тестовый набор 2,6/14/2021,3/22/2022,281 days,30-39,ж


In [25]:
fin = fin.drop([6694, 6703, 10280, 10281, 23078], axis=0)

In [26]:
fin.loc[(fin['COURSE'] == 'тестовый набор') | (fin['COURSE'] == 'тестовый набор 2')]

,ID,AGE,CITY,BU,COURSE,SD,ED,DURATION,AGE_RANGE,GENDER


In [ ]:
fin.to_csv('neto_stud_final.csv', encoding='cp1251')

********

1 - 'Нашёл новую работу по новой специальности'

2 - 'Стал фрилансером / нашёл новые проекты'

3 - 'Повысили на прежней работе'

4 - 'Усилил свои профессиональные навыки / Стало легче выполнять свою работу'

5 - 'Изменений не произошло'

6 - 'Другое (что именно)'

In [16]:
df_1=pd.read_csv('Карьерные изменения.csv')
df_1.columns=['ID', 'Reply']

In [21]:
df_1['Reply'].unique()

array(['Нашёл новую работу по новой специальности',
       'Стал фрилансером / нашёл новые проекты', 'Другое (что именно)',
       'Повысили на прежней работе', 'Изменений не произошло',
       'Усилил свои профессиональные навыки / Стало легче выполнять свою работу'],
      dtype=object)

In [23]:
df_1

,ID,Reply
0,6525677,Нашёл новую работу по новой специальности
1,6619787,Стал фрилансером / нашёл новые проекты
2,6844099,Другое (что именно)
3,6773732,Повысили на прежней работе
4,6698991,Изменений не произошло
...,...,...
2637,7226015,Другое (что именно)
2638,7251509,Другое (что именно)
2639,7254170,Другое (что именно)
2640,7268572,Другое (что именно)
